Ladies and gentlemen, we are gathered here today.... for some backtests 😎

In [ ]:
%load_ext IPython.extensions.autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../..')
from model import FinData
from model import train_valid_split
from model import CatboostFinModel

import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import datetime as dt

Пример, когда самому не надо задавать отдельно train, val и test датасеты, а просто задать другие параметры и функция всё обрежет сама 

In [ ]:
trading = ['Yandex', 'VTB', 'Sber', 'Rosneft', 'VK', 'T', 'Rusal', 'Rostelekom', 'Rosneft', 'Moex', 'Nornickel', 'MTS', 'Gazprom', 'Bashneft']
profit_list = []

for stock in trading:

    print("Stock is", stock)

    data = FinData("../../datasets/" + stock + '_10_min.csv')
    data.insert_shifts_norms([3, 6, 18])
    data.insert_rolling_means()
    data.insert_exp_rolling_means()
    data.insert_stochastic_oscillator()
    data.insert_high_low_diff()

    args = {"iterations" : 10000, 
                "depth" : 5, 
                "learning_rate" : 0.01,
                "use_best_model" : True, 
                "l2_leaf_reg" : 200,
                "verbose" : False, 
                "loss_function" : 'CrossEntropy', 
                "eval_metric" : 'Accuracy', 
                "cat_features" : data.cat_features, 
                "random_state" : 42,
                "early_stopping_rounds" : 2000}
    model = CatboostFinModel(args)

    st_dt = dt.datetime(2024, 1, 1)
    end_dt = dt.datetime(2024, 12, 31)

    test_st_dt = None
    test_end_dt = None

    profit = model.test_trading(data.df, start_date=st_dt, end_date = end_dt, test_st_dt=test_st_dt, test_end_dt=test_end_dt,
                                  initial_budget = 10000, cat=data.cat_features, num=data.numeric_features, commision = 0.0001, proportion=[1,  1, 1])

    profit_list.append(intraday)

Пример, когда задаются кастомные датасеты